In [7]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [19]:
url = f"https://shop.supervalu.ie/shopping/search/allaisles?q=milk"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [33]:
product_divs = soup.find_all("div", {"data-product": True})
result = pd.DataFrame()
for product_div in product_divs:
    product_data = json.loads(product_div["data-product"])
    # product_data_df = pd.json_normalize(product_data)
    #print(pd.DataFrame.from_dict(product_data))
    #df = pd.DataFrame(list(product_data.items()),columns = list(product_data.keys())) 
    print(pd.json_normalize(product_data))
    pd.concat([result,pd.json_normalize(product_data)])


           id                       name price     brand             category
0  1005787000  Avonmore Fresh Milk (2 L)  2.49  Avonmore  Milk, Butter & Eggs
           id                        name price      brand  \
0  1009043000  SuperValu Fresh Milk (3 L)  3.20  SuperValu   

              category  
0  Milk, Butter & Eggs  
           id                               name price     brand  \
0  1018392000  Avonmore Low Fat Super Milk (1 L)  1.65  Avonmore   

              category  
0  Milk, Butter & Eggs  
           id                        name price      brand  \
0  1025460000  SuperValu Fresh Milk (2 L)  2.29  SuperValu   

              category  
0  Milk, Butter & Eggs  
           id                              name price     brand  \
0  1020370000  Avonmore Light 1% Fat Milk (2 L)  2.49  Avonmore   

              category  
0  Milk, Butter & Eggs  
           id                                name price      brand  \
0  1291904000  SuperValu Fresh Skimmed Milk (1 L)  1

In [35]:
result.head()

""


In [9]:
url = f"https://www.tesco.ie/groceries/en-IE/search?query=milk"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Set up the Selenium driver
service = Service("path/to/chromedriver")
driver = webdriver.Chrome(service=service)

# Navigate to the Tesco grocery search page
driver.get("https://www.tesco.ie/groceries/en-IE/search?query=milk")

# Wait for the page to load and get the HTML
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.XPATH, "//li[@class='product-list--list-item']")))
html = driver.page_source

# Parse the HTML with Beautiful Soup
soup = BeautifulSoup(html, "html.parser")

# Find all product containers
product_containers = soup.find_all("li", {"class": "product-list--list-item"})




In [4]:
product_containers

[<li class="product-list--list-item first"><div class="styles__StyledVerticalTileWrapper-dvv1wj-0 dtCNPH"><div class="styles__StyledVerticalTile-dvv1wj-1 fZWbCY" data-auto="product-tile" data-auto-available="true" data-auto-is-restricted-disabled="false" data-auto-is-substitute="false" data-auto-type="Quantity"><div class="styles__StyledTileContentWrapper-dvv1wj-2 bkNhNP beans"><div class="styles__StyledTiledContent-dvv1wj-3 dAeIHg" data-auto-id="292769732" id="tile-292769732"><a aria-hidden="true" class="product-image-wrapper" href="/groceries/en-IE/products/292769732" tabindex="1"><div class="product-image__container"><picture><img alt="Tesco Fresh Milk1ltr" class="styled__Image-sjvkdn-0 bJErKA product-image beans-responsive-image__image" src="https://digitalcontent.api.tesco.com/v2/media/ghs/747525ca-092f-4e71-9018-13dc375a7c60/snapshotimagehandler_1949915135.jpeg?h=225&amp;w=225" srcset="https://digitalcontent.api.tesco.com/v2/media/ghs/747525ca-092f-4e71-9018-13dc375a7c60/snapshot

### References Taken:
1) Class Notebooks for BeautifulSoup
2) Youtube Video for Selenium: https://www.youtube.com/watch?v=UOsRrxMKJYk